<a href="https://colab.research.google.com/github/trDalmi/Data-Science-Portfolio/blob/main/notebookab0adcd08c.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# IMPORTANT: SOME KAGGLE DATA SOURCES ARE PRIVATE
# RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES.
import kagglehub
kagglehub.login()


In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

anzotr5_detecting_lane_path = kagglehub.dataset_download('anzotr5/detecting-lane')

print('Data source import complete.')


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/detecting-lane/192357-892475199_small.mp4
/kaggle/input/detecting-lane/test2.mp4


In [ ]:
import cv2

In [ ]:
def canny(img):
  gray =cv2.cvtColor(img,cv2.COLOR_RGB2GRAY)
  kernel = 5
  blur = cv2.GaussianBlur(gray,(kernel,kernel),0)
  canny = cv2.Canny(gray,50,150)
  return canny

In [ ]:
def region_of_intererst(image):
  height = image.shape[0]
  polygons = np.array([[(200,height),(1100,height),(550,250)]])
  mask = np.zeros_like(image)           # full filled black sheet of img np.size
  cv2.fillPoly(mask,np.array(polygons),225)          # drawing a polygon on the filter with white infill.
  masked_image = cv2.bitwise_and(image,mask)    # Concatenating the mask with the original image using and operator
  return masked_image

In [ ]:
def make_points(image,line):
  slope,intercept = line
  y1 = int(image.shape[0])
  y2 = int(y1*3.0/5)
  x1 = int((y1-intercept)/slope)
  x2 = int((y2-intercept)/slope)
  return [[x1,y1,x2,y2]]

In [ ]:
def average_slope_intercept(image,lines):
  left_fit = []
  right_fit = []
  if lines is None:
    return None
  for line in lines:
    for x1,y1,x2,y2 in line:
      fit = np.polyfit((x1,x2),(y1,y2),1)
      slope = fit[0]
      intercept = fit[1]
      if slope < 0:
        left_fit.append((slope,intercept))
      else:
        right_fit.append((slope,intercept))
  left_fit_average = np.average(left_fit,axis=0)
  right_fit_average = np.average(right_fit,axis=0)
  left_line = make_points(image,left_fit_average)
  right_line = make_points(image,right_fit_average)
  averaged_lines = [left_line,right_line]
  return averaged_lines

In [ ]:
def display_lines(img,lines):
  line_image = np.zeros_like(img)
  if lines is not None:
    for line in lines:
      for x1,y1,x2,y2 in line:
        cv2.line(line_image,(x1,y1),(x2,y2),(255,0,0),10)
    return line_image

In [ ]:
video_path = "/kaggle/input/detecting-lane/test2.mp4"

In [ ]:
cap = cv2.VideoCapture(video_path)

In [ ]:
while(cap.isOpened()):
      _, frame = cap.read()
      canny_image = canny(frame)
      cropped_image = region_of_intererst(canny_image)
      lines = cv2.HoughLinesP(cropped_image,2,np.pi/180, 100 ,np.array([]),minLineLength=40,maxLineGap=5)
      averaged_lines = average_slope_intercept(frame,lines)
      line_image = display_lines(frame,averaged_lines)
      combo_image = cv2.addWeighted(frame,0.8,line_image,1,1)
      cv2.imshow("result",combo_image)

      if cv2.waitKey(1) & 0xFF == ord('q'):  # Break the loop if 'q' is pressed
        break

cap.release()
cv2.destroyAllWindows()